In [1]:
import requests,time,os,csv,re,logging
from bs4 import BeautifulSoup

url_base='http://sh.lianjia.com/ershoufang/d'
date=time.strftime('%Y%m%d')

In [2]:
log_dir=os.path.join(os.getcwd(),'results/get_index.log')
logging.basicConfig(filename=log_dir,level=logging.INFO)

In [3]:
def get_html(url):
    try:
        r=requests.get(url,timeout=16)
        status=r.status_code
        r.raise_for_status()
        r.encoding=r.apparent_encoding
        return r.text
    except:
        now_time=time.strftime('%Y%m%d%H%M%S')
        logging.info(now_time +'      ' +'get_html 异常, ' '    url：'+ url)   

In [4]:
def parse_index_page(date,index_page):
    path=os.path.join(os.getcwd(),'results/index_info.csv')
    fieldnames=['date','total_num','on_sale_num','saled_in_90d','last_day_viewed',
                'key','title','total_price','total_price_unit','per_price','per_price_unit','prop1']
    if  not os.path.exists(path):     #如果文件不存在，创建文件并写入表头
        csv_file=open(path,'w',newline='')
        writer=csv.DictWriter(csv_file,fieldnames=fieldnames)
        writer.writeheader()
        csv_file.close()
        
    csv_file=open(path,'a+',newline='')
    writer=csv.DictWriter(csv_file,fieldnames=fieldnames)
    
    item={}
    
    soup=BeautifulSoup(index_page,'html.parser')
    
    #get_basic_info
    total_num=soup.find('div',attrs={'class':'search-result'}).span.string
    strong_num=soup.find_all('span',attrs={'class':'num strong-num'})
    on_sale_num=strong_num[0].string                #在售总数
    saled_in_90d=strong_num[1].string               #近90天成交
    last_day_viewed=strong_num[2].string            #昨天带看次数
    
    
    
    #update_basic_info
    item.update({'date':date,'total_num':total_num,'on_sale_num':on_sale_num,'saled_in_90d':saled_in_90d,
                'last_day_viewed':last_day_viewed})
    
    
    index_info=soup.find_all('a',attrs={'class':'text link-hover-green js_triggerGray js_fanglist_title'})
    for a in index_info:
        key=a['key']
        title=a['title']
        item.update({'key':key,'title':title})
        
        price_div=a.parent.next_sibling.next_sibling
        total_price=price_div.find('span',attrs={'class':'total-price strong-num'}).string
        total_price_unit=price_div.find('span',attrs={'class':'unit'}).string
        per_price_info=price_div.find('span',attrs={'class':'info-col price-item minor'}).string
        per_price_info=re.search('(\d+)(\S{3})',per_price_info).groups()
        per_price=per_price_info[0]
        per_price_unit=per_price_info[1]
        
        prop_div=price_div.next_sibling.next_sibling
        props=prop_div.find_all('span',attrs={'class':'c-prop-tag2'})
        
        if len(props)>0:
            if "距离" in props[0].string:
                prop1=props[0].string
            else:
                prop1=""
        else:
            prop1=""
            
#         prop2=props[1].string
#         prop3=props[2].string
        
        item.update({'total_price':total_price,'total_price_unit':total_price_unit,'per_price':per_price,
                    'per_price_unit':per_price_unit,'prop1':prop1})
        
        writer.writerow(item)    #定义writer2csv函数，每次调用函数写入item,结果是重复写入最后一个item，改用writer写入，无问题
        
        
    csv_file.close()

In [5]:
def parse_detail_page(detail_page):
    pass

In [36]:
def get_all_items():
    sleep_time=5
    for i in range(1,101):
        url=url_base+str(i)
        index_page=get_html(url)
        if index_page!=None：
            parse_index_page(date,total_num,index_page)
            time.sleep(sleep_time)
            print(i)
            sleep_time+=0.1

In [14]:
def try_agian(error_list):
    sleep_time=5
    new_error_list=[]
    for i in error_list:
        url=url_base+str(i)
        index_page=get_html(url)
        if index_page!=None:
            parse_index_page(date,index_page)
            time.sleep(sleep_time)
            sleep_time+=0.1
        else:
            new_error_list.append(i)
            now_time=time.strftime('%Y%m%d%H%M%S')
            logging.info(now_time +'      ' +'get None,  i=：'+ i)
        
    return new_error_list

In [9]:
sleep_time=5
error_list=[]
for i in range(1,101):
    url=url_base+str(i)
    index_page=get_html(url)
    if index_page!=None:
        parse_index_page(date,index_page)
        time.sleep(sleep_time)
        print(i)
        sleep_time+=0.1
    else:
        error_list.append(i)
        now_time=time.strftime('%Y%m%d%H%M%S')
        logging.info(now_time +'      ' +'get None,  i=：'+ i)  

#存在获取异常的页面，重试一次
if len(error_list)>0:
    new_error_list=try_agian(error_list)
    print('new_error_list:',new_error_list)

1
2
3
4
5
7
8
9
10
11
12
13
14
15
16
17
18
19
20
23
24
25
27
28
29
30
31
32
33
34
35
36
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
99
100
